### The list of postings needs to be cleaned to include the following:
1. bioguide ID
2. chamber
3. congress_number (verified for 86-118)
4. filter job_type == CongressMemberJob (column can be removed)
5. job_start_date (account for partial terms)
6. job_end_date (account for partial terms)
7. region_type
8. region_code
9. party_name

In [1]:
# Import Dependencies
import pandas as pd
from datetime import datetime
import csv
import re

In [2]:
# Load the postings CSV file into a DataFrame
postings_df = pd.read_csv('../data/raw/postings_raw.csv', dtype=str)
postings_df.head()

,bioguide_id,chamber,job_type,congress_number,congress_start_date,congress_end_date,region_type,region_code,party_name,job_start_date,job_end_date,party_start_date,party_end_date
0,A000002,Representative,CongressMemberJob,86,1959-01-03,1961-01-03,StateRegion,VA,Democrat,NaN,NaN,NaN,NaN
1,A000016,Representative,CongressMemberJob,86,1959-01-03,1961-01-03,StateRegion,MS,Democrat,NaN,NaN,NaN,NaN
2,A000024,Representative,CongressMemberJob,86,1959-01-03,1961-01-03,StateRegion,IN,Republican,NaN,NaN,NaN,NaN
3,A000054,Representative,CongressMemberJob,86,1959-01-03,1961-01-03,StateRegion,NJ,Democrat,NaN,NaN,NaN,NaN
4,A000062,Senator,CongressMemberJob,86,1959-01-03,1961-01-03,StateRegion,VT,Republican,NaN,NaN,NaN,NaN


In [3]:
# Display the shape of the DataFrame
postings_df.shape

(55071, 13)

18 members had switched parties while in office. A second mongodb extraction logic was needed to expand each posting into separate ones with a single party. 

In [4]:
# display the unique values in party_name column
# we should find only strings and no lists of multiple parties
postings_df['party_name'].unique()

array(['Democrat', 'Republican', 'Independent Democrat',
       'Popular Democrat', nan, 'New Progressive', 'Conservative',
       'Independent', 'Independence Party (Minnesota)', 'Libertarian'],
      dtype=object)

In [5]:
# Count the number of duplicate rows
duplicate_count = postings_df.duplicated().sum()
print(f'Number of duplicate rows: {duplicate_count}')

Number of duplicate rows: 36714


In [6]:
# Remove duplicate rows
postings_df = postings_df.drop_duplicates()
postings_df.shape

(18357, 13)

In [7]:
# List unique values in the job_type column
postings_df['job_type'].unique()


array(['CongressMemberJob', 'CongressLeadershipJob', 'OtherJob'],
      dtype=object)

In [8]:
# Filter job_type to include only "CongressMemberJob"
postings_df = postings_df[postings_df['job_type'] == 'CongressMemberJob']
postings_df.shape

(18228, 13)

In [9]:
# remove the JobType column
postings_df = postings_df.drop(columns=['job_type'])
postings_df.shape

(18228, 12)

In [10]:
# display the unique values in party_name column
# verifying there are no rows with Nan values
postings_df['party_name'].unique()

array(['Democrat', 'Republican', 'Independent Democrat',
       'Popular Democrat', nan, 'New Progressive', 'Conservative',
       'Independent', 'Independence Party (Minnesota)', 'Libertarian'],
      dtype=object)

In [11]:
# Remove rows with NaN values in party_name column
postings_df = postings_df.dropna(subset=['party_name'])
postings_df['party_name'].unique()

array(['Democrat', 'Republican', 'Independent Democrat',
       'Popular Democrat', 'New Progressive', 'Conservative',
       'Independent', 'Independence Party (Minnesota)', 'Libertarian'],
      dtype=object)

A data error was found during exploration. The start date of the first congress served would be found in the start date of the final term served. These datapoints need to be removed.

In [12]:
# Check the datatypes in the date columns
date_columns = ['job_start_date', 'job_end_date', 'congress_start_date', 'congress_end_date', 'party_start_date', 'party_end_date']
for col in date_columns:
    postings_df[col] = pd.to_datetime(postings_df[col], errors='coerce')
    print(f"{col} dtype: {postings_df[col].dtype}")

job_start_date dtype: datetime64[ns]
job_end_date dtype: datetime64[ns]
congress_start_date dtype: datetime64[ns]
congress_end_date dtype: datetime64[ns]
party_start_date dtype: datetime64[ns]
party_end_date dtype: datetime64[ns]


In [13]:
postings_df.head()

,bioguide_id,chamber,congress_number,congress_start_date,congress_end_date,region_type,region_code,party_name,job_start_date,job_end_date,party_start_date,party_end_date
0,A000002,Representative,86,1959-01-03,1961-01-03,StateRegion,VA,Democrat,NaT,NaT,NaT,NaT
1,A000016,Representative,86,1959-01-03,1961-01-03,StateRegion,MS,Democrat,NaT,NaT,NaT,NaT
2,A000024,Representative,86,1959-01-03,1961-01-03,StateRegion,IN,Republican,NaT,NaT,NaT,NaT
3,A000054,Representative,86,1959-01-03,1961-01-03,StateRegion,NJ,Democrat,NaT,NaT,NaT,NaT
4,A000062,Senator,86,1959-01-03,1961-01-03,StateRegion,VT,Republican,NaT,NaT,NaT,NaT


In [14]:
# Make a list of bioguide_ids where the job_start_date is earlier than the congress_start_date
invalid_date_bioguide_ids = postings_df[pd.to_datetime(postings_df['job_start_date']) < pd.to_datetime(postings_df['congress_start_date'])]['bioguide_id'].unique().tolist()
# print the list
print(invalid_date_bioguide_ids)
# print a count of how many bioguide_ids are in the list
print(f"Number of bioguide_ids with invalid date ranges: {len(invalid_date_bioguide_ids)}")


['P000152', 'B001251', 'B001270', 'D000482', 'G000582', 'M001165']
Number of bioguide_ids with invalid date ranges: 6


In [15]:
# replace the job_start_date with the congress_start_date for the invalid_date_bioguide_ids
for bioguide_id in invalid_date_bioguide_ids:
    congress_start_date = postings_df.loc[postings_df['bioguide_id'] == bioguide_id, 'congress_start_date'].values[0]
    postings_df.loc[postings_df['bioguide_id'] == bioguide_id, 'job_start_date'] = congress_start_date


In [16]:
# subtract one day from the congress_end_date to prevent overlap with the job_start_date
postings_df['congress_end_date'] = postings_df['congress_end_date'] - pd.Timedelta(days=1)
# Verify the change
postings_df['congress_end_date'].head()


0   1961-01-02
1   1961-01-02
2   1961-01-02
3   1961-01-02
4   1961-01-02
Name: congress_end_date, dtype: datetime64[ns]

In [17]:
postings_df.head()

,bioguide_id,chamber,congress_number,congress_start_date,congress_end_date,region_type,region_code,party_name,job_start_date,job_end_date,party_start_date,party_end_date
0,A000002,Representative,86,1959-01-03,1961-01-02,StateRegion,VA,Democrat,NaT,NaT,NaT,NaT
1,A000016,Representative,86,1959-01-03,1961-01-02,StateRegion,MS,Democrat,NaT,NaT,NaT,NaT
2,A000024,Representative,86,1959-01-03,1961-01-02,StateRegion,IN,Republican,NaT,NaT,NaT,NaT
3,A000054,Representative,86,1959-01-03,1961-01-02,StateRegion,NJ,Democrat,NaT,NaT,NaT,NaT
4,A000062,Senator,86,1959-01-03,1961-01-02,StateRegion,VT,Republican,NaT,NaT,NaT,NaT


In [18]:
# using if statments, create a new column called posting_start_date and set to the value frome the following logic
# Use party_start_date if not null, else use job_start_date if not null, then use congress_start_date
def determine_posting_start_date(row):
    if pd.notnull(row['party_start_date']):
        return row['party_start_date']
    elif pd.notnull(row['job_start_date']):
        return row['job_start_date']
    else:
        return row['congress_start_date']
    
postings_df['posting_start_date'] = postings_df.apply(determine_posting_start_date, axis=1)
postings_df.head(10)

,bioguide_id,chamber,congress_number,congress_start_date,congress_end_date,region_type,region_code,party_name,job_start_date,job_end_date,party_start_date,party_end_date,posting_start_date
0,A000002,Representative,86,1959-01-03,1961-01-02,StateRegion,VA,Democrat,NaT,NaT,NaT,NaT,1959-01-03
1,A000016,Representative,86,1959-01-03,1961-01-02,StateRegion,MS,Democrat,NaT,NaT,NaT,NaT,1959-01-03
2,A000024,Representative,86,1959-01-03,1961-01-02,StateRegion,IN,Republican,NaT,NaT,NaT,NaT,1959-01-03
3,A000054,Representative,86,1959-01-03,1961-01-02,StateRegion,NJ,Democrat,NaT,NaT,NaT,NaT,1959-01-03
4,A000062,Senator,86,1959-01-03,1961-01-02,StateRegion,VT,Republican,NaT,NaT,NaT,NaT,1959-01-03
5,A000073,Representative,86,1959-01-03,1961-01-02,StateRegion,OK,Democrat,NaT,NaT,NaT,NaT,1959-01-03
6,A000094,Representative,86,1959-01-03,1961-01-02,StateRegion,NC,Democrat,NaT,NaT,NaT,NaT,1959-01-03
7,A000105,Representative,86,1959-01-03,1961-01-02,StateRegion,AR,Independent Democrat,NaT,NaT,NaT,NaT,1959-01-03
8,A000106,Representative,86,1959-01-03,1961-01-02,StateRegion,TX,Republican,NaT,NaT,NaT,NaT,1959-01-03
9,A000138,Representative,86,1959-01-03,1961-01-02,DistrictRegion,IL,Republican,1959-01-03,NaT,NaT,NaT,1959-01-03


In [19]:
# Repeat the process for posting_end_date
def determine_posting_end_date(row):
    if pd.notnull(row['party_end_date']):
        return row['party_end_date']
    elif pd.notnull(row['job_end_date']):
        return row['job_end_date']
    else:
        return row['congress_end_date']
postings_df['posting_end_date'] = postings_df.apply(determine_posting_end_date, axis=1)
postings_df.head(10)

,bioguide_id,chamber,congress_number,congress_start_date,congress_end_date,region_type,region_code,party_name,job_start_date,job_end_date,party_start_date,party_end_date,posting_start_date,posting_end_date
0,A000002,Representative,86,1959-01-03,1961-01-02,StateRegion,VA,Democrat,NaT,NaT,NaT,NaT,1959-01-03,1961-01-02
1,A000016,Representative,86,1959-01-03,1961-01-02,StateRegion,MS,Democrat,NaT,NaT,NaT,NaT,1959-01-03,1961-01-02
2,A000024,Representative,86,1959-01-03,1961-01-02,StateRegion,IN,Republican,NaT,NaT,NaT,NaT,1959-01-03,1961-01-02
3,A000054,Representative,86,1959-01-03,1961-01-02,StateRegion,NJ,Democrat,NaT,NaT,NaT,NaT,1959-01-03,1961-01-02
4,A000062,Senator,86,1959-01-03,1961-01-02,StateRegion,VT,Republican,NaT,NaT,NaT,NaT,1959-01-03,1961-01-02
5,A000073,Representative,86,1959-01-03,1961-01-02,StateRegion,OK,Democrat,NaT,NaT,NaT,NaT,1959-01-03,1961-01-02
6,A000094,Representative,86,1959-01-03,1961-01-02,StateRegion,NC,Democrat,NaT,NaT,NaT,NaT,1959-01-03,1961-01-02
7,A000105,Representative,86,1959-01-03,1961-01-02,StateRegion,AR,Independent Democrat,NaT,NaT,NaT,NaT,1959-01-03,1961-01-02
8,A000106,Representative,86,1959-01-03,1961-01-02,StateRegion,TX,Republican,NaT,NaT,NaT,NaT,1959-01-03,1961-01-02
9,A000138,Representative,86,1959-01-03,1961-01-02,DistrictRegion,IL,Republican,1959-01-03,NaT,NaT,NaT,1959-01-03,1961-01-02


In [20]:
# Check for null values in posting_start_date and posting_end_date
null_start_dates = postings_df[postings_df['posting_start_date'].isnull()]
null_end_dates = postings_df[postings_df['posting_end_date'].isnull()]
print(f"Number of rows with null posting_start_date: {null_start_dates.shape[0]}")
print(f"Number of rows with null posting_end_date: {null_end_dates.shape[0]}")

Number of rows with null posting_start_date: 0
Number of rows with null posting_end_date: 0


In [21]:
# remove unnecessary date columns
columns_to_remove = ['job_start_date', 'job_end_date', 'congress_start_date', 'congress_end_date', 'party_start_date', 'party_end_date']
postings_df = postings_df.drop(columns=columns_to_remove)
postings_df.head()

,bioguide_id,chamber,congress_number,region_type,region_code,party_name,posting_start_date,posting_end_date
0,A000002,Representative,86,StateRegion,VA,Democrat,1959-01-03,1961-01-02
1,A000016,Representative,86,StateRegion,MS,Democrat,1959-01-03,1961-01-02
2,A000024,Representative,86,StateRegion,IN,Republican,1959-01-03,1961-01-02
3,A000054,Representative,86,StateRegion,NJ,Democrat,1959-01-03,1961-01-02
4,A000062,Senator,86,StateRegion,VT,Republican,1959-01-03,1961-01-02


In [22]:
# Rename posting_start_date to start_date and posting_end_date to end_date
postings_df = postings_df.rename(columns={'posting_start_date': 'start_date',
                                          'posting_end_date': 'end_date',
                                          'party_name': 'party'})
postings_df.head()

,bioguide_id,chamber,congress_number,region_type,region_code,party,start_date,end_date
0,A000002,Representative,86,StateRegion,VA,Democrat,1959-01-03,1961-01-02
1,A000016,Representative,86,StateRegion,MS,Democrat,1959-01-03,1961-01-02
2,A000024,Representative,86,StateRegion,IN,Republican,1959-01-03,1961-01-02
3,A000054,Representative,86,StateRegion,NJ,Democrat,1959-01-03,1961-01-02
4,A000062,Senator,86,StateRegion,VT,Republican,1959-01-03,1961-01-02


In [23]:
# Cleanup the region_type column
# find the unique values in the region_type column
postings_df['region_type'].unique()

array(['StateRegion', 'DistrictRegion', 'TerritoryRegion'], dtype=object)

In [24]:
# Rename values in region_type column
# StateRegion and DistrictRegion == Stae
# TerritoryRegion == Territory
postings_df['region_type'] = postings_df['region_type'].replace({'DistrictRegion': 'State',
                                                                 'StateRegion': 'State',
                                                                 'TerritoryRegion': 'Territory'})

In [25]:
# Trust by verify
# find the unique values in the region_type column
postings_df['region_type'].unique()

array(['State', 'Territory'], dtype=object)

In [26]:
# save as a new CSV file
postings_df.to_csv('../data/processed/postings_cleaned.csv', index=False)

In [27]:
# Set date columns to strings
date_columns = ['start_date', 'end_date']
for col in date_columns:
    postings_df[col] = postings_df[col].dt.strftime('%Y-%m-%d')
# Save as a .xlsx file
import openpyxl
postings_df.to_excel('../data/processed/postings_cleaned.xlsx', index=False)